# Laboratorio 1

In [ ]:
#Instalcion de todas las librerias
import pandas as pd
import numpy as np
import sys
from sklearn.base import BaseEstimator, ClassifierMixin
import matplotlib.pyplot as plt
import seaborn as sb


: 

### Carga de datos

In [ ]:
# Uso de la libreria pandas para la lectura de archivos
data=pd.read_csv('train_data.csv', sep=',', encoding = 'utf-8')
# Asignación a una nueva variable de los datos leidos
data_t=data

In [ ]:
data_t

## Entendimiento de los Datos

Antes de comenzar a analizar los datos, es fundamental identificar cuáles son más importantes que otros.

###  Variables Necesarias

- **ra (Ascensión recta)**
  - Indica la posición del objeto en el cielo.
- **dec (Declinación)**
  - Complementa la posición del objeto en coordenadas celestes.
- **u, g, r, z, i (Magnitudes en diferentes filtros)**
  - Permiten clasificar y caracterizar los objetos en función de su emisión de luz en distintos rangos de longitud de onda.
  - Son fundamentales para estimar redshifts fotométricos si se quiere hacer una estimación sin espectroscopía.
- **class**
  - Indica si el objeto es una estrella (**STAR**), una galaxia (**GALAXY**) o un cuásar (**QSO**).
  - Importante porque el redshift se aplica principalmente a galaxias y cuásares.
- **redshift**
  - Es el valor clave de estudio, ya que representa el corrimiento al rojo del objeto.


###  Variables no necesarias

- **mjd (Día juliano)**
  - Puede ser útil si se estudian variaciones en observaciones a lo largo del tiempo.
- **run**
    - Es la cantidad de observaciones realizadas por el telescopio pero en este caso no es importante
- **camcol**
    - Columna de la cámara en la que se registró el objeto. Se reviso la correlacion lineal pero fue muy bajo tambien por eso se descarto esto.     
 - **field**
    - Sección de la columna de la cámara en la que se registró el objeto. Ya que Camcol no la tomamos importante esta tampoco para el analisis      
- **score**
  - Indica la calidad de la observación. Podría ser útil para filtrar datos de baja calidad.
- **clean**
  - También puede ayudar a filtrar observaciones con problemas en la fotometría.
- **rowv y colv**
  - Indican velocidades en grados/día, que pueden ser útiles si se quiere analizar movimiento propio, pero no son necesarias para un análisis del redshift.




In [ ]:
outliers3 = data_t[(data_t['score'] < 0.4) & (data_t["clean"] == 0)]
outliers3


In [ ]:
#Entendimiento del tamaño del data Set a ser manejado
print('Dimensiones de los datos: ', data_t.shape, 'El primer valor corresponde a filas y el segundo a columnas')

In [ ]:
# Visualizacion de los primeros 10 datos
display(data_t.head(10))

In [ ]:
# Imprimimos los diferentes tipos de las columnas
data_t.dtypes

In [ ]:
# Imprimir las columnas numericas
columnasNumericas= data_t.select_dtypes(include=['int','float']).columns
print(columnasNumericas)

In [ ]:
data_t.describe( include='all')

### Ahora vamos a empezar a revisar los datos que se encuentran mal de cada columna en base a sus datos para ver si se pueden corregir o deben de ser eliminados

In [ ]:
#Para empezar revisamos si hay valores nulos en el dataset, pero no se encuentran datos nulos
data_t.isnull().sum()

In [ ]:
# Ver las filas duplicadas
duplicados = data_t[data_t.duplicated()]

# Para ver tanto las filas originales como sus duplicados:
duplicados_todos = data_t[data_t.duplicated(keep=False)]

# Para ver cuántas filas duplicadas hay:
num_duplicados = data_t.duplicated().sum()

# Para ver un resumen más detallado:
print(f"Total de filas duplicadas: {data_t.duplicated().sum()}")


### Revisando la Columna 'ra' la cual describe Ubicación del objeto en su componente de ascensión recta de coordenada celeste.

In [ ]:
data['ra'].describe()

In [ ]:
#Revisar si hay datos Nulos en la columna ra
data['ra'].isna().sum()

In [ ]:
# Ya que Ra describe la ubicación del objeto en su componente de ascensión recta de coordenada celeste. Este valor debe de estar entre 0 y 360 grados.
df_out_rango = data_t[(data_t["ra"] < 0) | (data_t["ra"] > 360)]
print(df_out_rango)

In [ ]:
#Revisamos si se encuentra dentro del rango intercuartil, los valores que se encuentren por fuera de esto son outlayers [Q1−1.5×IQR,Q3+1.5×IQR]

Q1 = data_t['ra'].quantile(0.25)
Q3 = data_t['ra'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(Q1,Q3,IQR)
print(lower_bound,upper_bound) 
 
outliers = data_t[(data_t['ra'] < lower_bound) | (data_t['ra'] > upper_bound)]
# print(outliers['ra'])
outliers
plt.boxplot(data_t['ra'])
plt.title("Boxplot de la Ascensión Recta (ra)")
plt.ylabel("Ascensión Recta")
plt.show()

PENDIENTE PONER EN TIPO MARKDOWN
Distribución General:
La mediana (línea naranja) se encuentra cerca del centro del cajón, indicando que la distribución no está demasiado sesgada.
La mayoría de los datos se encuentran en el rango intercuartil (IQR), entre el primer y tercer cuartil.
Valores Atípicos (Outliers):
Se observan varios outliers en la parte inferior, lo que sugiere que hay valores significativamente bajos de ascensión recta en comparación con la mayoría de los datos.
Algunos de estos valores están bastante lejos del límite inferior del bigote, indicando que podrían ser errores de medición o registros especiales que deben analizarse más a fondo.
Límites de los Bigotes:
El límite inferior del bigote está alrededor de 90 pero hay valores por debajo de 50, que se consideran atípicos según el método del IQR.
El límite superior está alrededor de 250, sin outliers significativos en esta dirección, lo que sugiere que los valores altos no presentan anomalías.

In [ ]:
# Crear el histograma para visualizar la distribución de 'ra'
plt.figure(figsize=(8, 5))
plt.hist(data_t["ra"], bins=20, edgecolor="black", alpha=0.7, color="orange")
plt.xlabel("Ascensión Recta (ra)")
plt.ylabel("Frecuencia")
plt.title("Distribución de la Ascensión Recta")
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()

In [ ]:
# for col in data_t.select_dtypes(include=['object']):
#     print(f"Valores únicos en {col}:\n", data_t[col].unique())

## Columna dec
Describe: Ubicación del objeto en su componente de declinación de coordenada celeste.

In [ ]:
data_t['dec'].describe()

In [ ]:
#Ya que Dec describe la ubicación del objeto en su componente de declinación de coordenada celeste. Este valor debe de estar entre -90 y 90 grados.
out_rango = data_t[(data_t["dec"] < -90) | (data_t["dec"] > 90)]
print(out_rango)

In [ ]:
Q1 = data_t['dec'].quantile(0.25)
Q3 = data_t['dec'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(Q1,Q3,IQR)
print(lower_bound,upper_bound) 
 
outliers = data_t[(data_t['dec'] < lower_bound) | (data_t['dec'] > upper_bound)]
# print(outliers['dec'])
outliers
plt.boxplot(data_t['dec'])
plt.title("Boxplot de la Ascensión Recta (dec)")
plt.ylabel("Ascensión Recta")
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(data_t["dec"], bins=20, edgecolor="black", alpha=0.7, color="orange")
plt.xlabel("Ascensión Recta (dec)")
plt.ylabel("Frecuencia")
plt.title("Distribución de la Ascensión Recta")
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()

### Vamos a revisar ese dato que se sale de la asension de la recta normal

In [ ]:
# Valores a revisar ya que son valores lejos del centro
outliers2 = data_t[(data_t['dec'] > 10)]
outliers2

## Columna U
Magnitud del objeto capturada en el filtro ultravioleta. Valor mayor a 0.

In [ ]:
data_t['u'].describe()

In [ ]:
# Revisamos si es que existen valores por debajo del 0 
outliers = data_t[(data_t['u'] <= 0)]
outliers

In [ ]:
# Revisamos si es que existen valores por encima de 30, pq Normalmente, las magnitudes de objetos astronómicos en el filtro ultravioleta están en un rango entre -5 y 25.
outliers = data_t[(data_t['u'] > 30)]
outliers

In [ ]:
Q1 = data_t['u'].quantile(0.25)
Q3 = data_t['u'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(Q1,Q3,IQR)
print(lower_bound,upper_bound) 
 
outliers = data_t[(data_t['u'] < lower_bound) | (data_t['u'] > upper_bound)]
# print(outliers['dec'])
outliers
plt.boxplot(data_t['u'])
plt.title("Boxplot de la Ascensión Recta (u)")
plt.ylabel("Ascensión Recta")
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(data_t["u"], bins=20, edgecolor="black", alpha=0.7, color="orange")
plt.xlabel("Ascensión Recta (u)")
plt.ylabel("Frecuencia")
plt.title("Distribución de la Ascensión Recta")
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()

## Columna g
Describe: Ubicación del objeto en su componente de declinación de coordenada celeste.

In [ ]:
data_t['g'].describe()

In [ ]:
#Ya que g describe la magnitud del objeto en el filtro verde este va entre 12-25
out_rango = data_t[(data_t["g"] > 25) | (data_t["g"] < 12)]
print(out_rango)

In [ ]:
menor0 = data_t[(data_t["g"] <= 0)]
print(menor0)

In [ ]:
Q1 = data_t['g'].quantile(0.25)
Q3 = data_t['g'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(Q1,Q3,IQR)
print(lower_bound,upper_bound) 
 
outliers = data_t[(data_t['g'] < lower_bound) | (data_t['g'] > upper_bound)]
# print(outliers['g'])
outliers
plt.boxplot(data_t['g'])
plt.title("Boxplot de la Ascensión Recta (g)")
plt.ylabel("Ascensión Recta")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(data_t["g"], bins=20, edgecolor="black", alpha=0.7, color="orange")
plt.xlabel("Ascensión Recta (g)")
plt.ylabel("Frecuencia")
plt.title("Distribución de la Ascensión Recta")
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()

## Columna r
Describe: Magnitud del objeto capturada en el filtro rojo. Valor mayor a 0.

In [ ]:
data_t['r'].describe()

In [ ]:
menor0 = data_t[(data_t["r"] <= 0)]
print(menor0)

## Columna z
Describe: Magnitud del objeto capturada en el filtro casi-infrarrojo. Valor mayor a 0.

In [ ]:
data_t['z'].describe()

## Columna i
Describe: Magnitud del objeto capturada en el filtro infrarrojo. Valor mayor a 0.

In [ ]:
data_t['i'].describe()

## Columna camcol
Describe: Columna de la cámara en la que se registró el objeto.

In [ ]:
print(data_t['camcol'].value_counts())
print("Valores únicos:", data_t['camcol'].unique())

#Podemos ver que es una columna numerica pero deberia de ser categorica solo toma valores 
# específicos como {1, 2, 3, 4, 5, 6}, es más una categoría que un número

In [ ]:
data_t['camcol'] = data_t['camcol'].astype('category')
print("Tipo de dato:", data_t['camcol'].dtype)

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(data_t["camcol"], bins=20, edgecolor="black", alpha=0.7, color="orange")
plt.xlabel("X")
plt.ylabel("Frecuencia")
plt.title("Distribución de la Ascensión Recta")
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()


## Columna field
Describe: Sección de la columna de la cámara en la que se registró el objeto.


In [ ]:
for col in data.columns:
    unique_values = data[col].nunique()
    print(f"{col}: {unique_values} valores únicos")


## Análisis de calidad de los datos

Completitud

In [ ]:
# Cálculo de valores nulos por columna
((data_t.isnull().sum()/data_t.shape[0])).sort_values(ascending=False)

Al hacer un examen de completitud de los datos podemos ver que no hay datos nulos en ninguna columna del data frame

2.2 Unicidad

In [ ]:
data_t.duplicated(keep = False).sum()
print(f"Filas duplicadas: {data_t.duplicated(keep=False).sum()}")

Se identificaron 34 filas duplicadas, ahora vamos a ver a detalle que es lo que está sucediendo

In [ ]:
duplicados = data_t.loc[data_t.duplicated(keep=False)]
duplicados

Viendo a detalle los datos se puede ver que hay filas completamente duplicadas. Analizando el contexto de los datos y la forma en que se toman no tiene sentido que hayan filas completamente duplicadas, por lo cual consideramos eliminarlas. Eliminaremos 17 filas ya que estas corresponden a las filas duplicadas

2.3 Consistencia 

2.4 Validez

## Visualización de datos

In [ ]:
data_t.hist(figsize=(15,18),bins=30,edgecolor="black")

Análisis de correlaciones

In [ ]:
data_t["class"].value_counts()

In [ ]:
data_t["class"] = data_t["class"].replace({
    "S": "STAR",
    "G": "GALAXY",
    "QUASAR": "QSO"
})
data_t["class"].value_counts()

In [ ]:
features=["ra","dec","u","g","r","z","i","class","redshift","mjd"]
data_t=data_t[features]
data_t.head()

One-Hot encoding

In [ ]:
dumies=pd.get_dummies(data_t["class"], dtype=int)
data_t=pd.concat([data_t, dumies], axis=1)

In [ ]:
data_t=data_t.drop(["class"],axis=1)

In [ ]:
data_t.head()

Análisis de correlaciones

In [ ]:
sb.set(rc={"figure.figsize":(25,12)})
sb.heatmap( data_t.corr(),annot=True, cmap="YlGnBu" )

### REGRESION LINEAL

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


X_train, X_test, y_train, y_test = train_test_split(data_t, data_t["redshift"], test_size=0.3, random_state=1)

X_test.shape, y_test.shape

In [ ]:


# Crear y entrenar el modelo
regression = LinearRegression()
regression.fit(X_train, y_train)